,kp0_x,kp0_y,kp0_z,kp1_x,kp1_y,kp1_z,kp2_x,kp2_y,kp2_z,kp4_x,kp4_y,kp4_z,kp5_x,kp5_y,kp5_z,kp6_x,kp6_y,kp6_z
0,-5.018395,-19.176620,3.059264,4.301794,-15.118471,4.847923,-4.452908,10.889791,4.315517,-15.216230,-18.742833,1.448757,12.297606,-3.303560,4.812236,16.330635,-5.288675,3.387822
1,18.028572,18.796394,0.697469,-13.179035,-0.192924,3.875664,-9.146039,-12.051373,3.116491,8.529791,5.456416,0.806106,15.843652,-11.115688,1.258911,-10.417524,5.292233,0.082939
2,9.279758,13.297706,1.460723,-17.397936,-18.624459,4.697495,13.149500,-19.779115,1.654490,10.431402,-7.425761,4.648488,-7.279861,-15.205385,2.486243,-14.204205,5.341188,2.560465
3,3.946339,-11.506436,1.831809,17.955421,16.372816,4.474137,-5.729867,12.618457,0.317792,2.451088,0.342828,4.040602,-15.597923,-6.495393,1.504392,-0.421890,1.430987,1.132479
4,-13.759254,-12.727001,2.280350,18.625281,-9.648801,2.989500,-8.762620,8.274294,1.554912,10.838687,16.302659,3.167019,-10.882593,17.716388,1.424202,19.426018,-16.388409,3.225864


In [ ]:
import numpy as np
import pandas as pd

def get_body_rotations(sorted_point_data):
    print('Processing to get body related rotation matrix ...... ')
    nf = len(sorted_point_data)  # number of frames

    r_roots = np.zeros((nf, 3, 3))  #
    r_root_inv = np.zeros((nf, 3, 3))  #
    r_root_inv_oriented = np.zeros((nf, 3, 3))  #
    dir_backs = np.zeros((nf, 3))  #
    r_roots[:] = np.nan
    r_root_inv[:] = np.nan
    r_root_inv_oriented[:] = np.nan
    dir_backs[:] = np.nan

    timeofheadnans = []
    # DO IT ALL FOR THE NEW ROOT NODE (ASSUMING BUTT TO NECK!)
    for t in np.arange(nf):
        if np.isnan(sorted_point_data[t, 4, 0]) or np.isnan(sorted_point_data[t, 6, 0]):  # 4 is neck and 6 is tail
            continue
    
        # THIS IS A STUPID WAY OF CREATING RROOTS (ANGLE OF BODY RELATIVE TO A WALL)
        # It should just be xdir with z=0...
        xdir = sorted_point_data[t, 4, :] - sorted_point_data[t, 6, :]  # from tail to neck
        xdir[2] = 0.
        ll = np.linalg.norm(xdir)
        if ll < 0.001:
            continue
        xdir = xdir / ll  # x direction of the animal from the top view (chosen)
        zdir = np.ravel(np.array([0, 0, 1]))
        ydir = np.cross(zdir, xdir)  # y direction according to right-hand rule
        shitinv = (np.array([xdir, ydir, zdir])) + 0.  # mapping from global coordinates to animal coordinates
        r_roots[t, :, :] = np.transpose(shitinv)  # mapping from animal to global from the top view
    
        # HERE IS THE REAL ROOT ANGLE!!!
        # SAME AS WITHLOCKINZ = TRUE AND WITHROTATIONS = TRUE
        xdir = sorted_point_data[t, 4, :] - sorted_point_data[t, 6, :]
        ll = np.linalg.norm(xdir)
        if ll < 0.001:
            continue
        xdir = xdir / ll  # unit vector from tail to neck in global coordinates (not top view)
        ydir = np.zeros(3)
        ydir[0] = -xdir[1] + 0.
        ydir[1] = xdir[0] + 0.
        ydir[2] = 0.
        ydir = ydir / np.linalg.norm(ydir)  # an orthogonal vector to the x dir in xy plane
        zdir = np.cross(xdir, ydir)
        zdir = zdir / np.linalg.norm(zdir)
        r_root_inv[t, :, :] = (np.array([xdir, ydir, zdir])) + 0.  # for ego3
    
        # HERE IS THE OTHER OTHER OTHER ROOT ANGLE!!!
        xdir = sorted_point_data[t, 4, :] - sorted_point_data[t, 6, :]
        xdir[2] = 0  # so we just keep the planar direction from the body coordinates!!!
        ll = np.linalg.norm(xdir)
        if ll < 0.001:
            timeofheadnans.append(t)
            continue
        xdir = xdir / ll
        ydir = np.zeros(3)
        ydir[0] = -xdir[1] + 0.
        ydir[1] = xdir[0] + 0.
        ydir[2] = 0.
        ydir = ydir / np.linalg.norm(ydir)
        zdir = np.zeros(3)
        zdir[2] = 1.
        r_root_inv_oriented[t, :, :] = (np.array([xdir, ydir, zdir])) + 0.  # for ego2
    
        # if( WITHLOCKEDINZ == False and WITHROTATIONS == True ):
        # xdir = sorted_point_data[t,4,:] - sorted_point_data[t,6,:]
        # ll = np.linalg.norm(xdir)
        # if(ll<0.001):
        # continue
        # xdir[2] = 0.
        # xdir = xdir / ll
        # ydir = zeros(3)
        # ydir[0] = -xdir[1]+0.
        # ydir[1] = xdir[0]+0.
        # ydir[2] = 0.
        # ydir = ydir/np.linalg.norm(ydir)
        # zdir = zeros(3)
        # zdir[2] = 1.
        # RrootINV = ( np.array([xdir, ydir, zdir]) ) + 0.
        if ~np.isnan(sorted_point_data[t, 6, 0]):
            dir_to_butt = sorted_point_data[t, 4, :] - sorted_point_data[t, 5, :]
            dir_to_butt = dir_to_butt / np.linalg.norm(dir_to_butt)
            dir_to_butt = np.dot(r_root_inv[t, :, :], dir_to_butt)
            dir_backs[t, :] = dir_to_butt + 0.
    
    return r_roots, r_root_inv_oriented, r_root_inv, dir_backs

def get_body_rotations_df_new(df, spineF_kp=4, tailB_kp=6, spineM_kp=5):
    print('Processing to get body related rotation matrix ......')
    
    nf = df.shape[0]
    r_roots = np.zeros((nf, 3, 3))  
    r_root_inv = np.zeros((nf, 3, 3))  
    r_root_inv_oriented = np.zeros((nf, 3, 3))  
    dir_backs = np.zeros((nf, 3))  

    r_roots[:] = np.nan
    r_root_inv[:] = np.nan
    r_root_inv_oriented[:] = np.nan
    dir_backs[:] = np.nan

    # Extract x, y, and z coordinates for relevant keypoints
    spineF_x, spineF_y, spineF_z = df[f"kp{spineF_kp}_x"], df[f"kp{spineF_kp}_y"], df[f"kp{spineF_kp}_z"]
    tailB_x, tailB_y, tailB_z = df[f"kp{tailB_kp}_x"], df[f"kp{tailB_kp}_y"], df[f"kp{tailB_kp}_z"]
    spineM_x, spineM_y, spineM_z = df[f"kp{spineM_kp}_x"], df[f"kp{spineM_kp}_y"], df[f"kp{spineM_kp}_z"]

    # Precompute valid mask
    valid_mask = ~np.isnan(spineF_x) & ~np.isnan(tailB_x) 

    # 1. NaN handling when computing xdir (body direction from tail to neck)
    xdir = np.column_stack((spineF_x - tailB_x, spineF_y - tailB_y, np.zeros_like(spineF_z)))
    xdir[~valid_mask] = np.nan  # Explicitly set NaN where mask is invalid
    ll = np.linalg.norm(xdir, axis=1, keepdims=True)
    valid_mask &= (ll[:, 0] >= 0.001)
    xdir[~valid_mask] = np.nan  # Propagate NaNs after length check
    xdir[valid_mask] /= ll[valid_mask]

    # Compute y-direction using right-hand rule
    zdir = np.array([0, 0, 1])
    ydir = np.cross(zdir, xdir)
    ydir[~valid_mask] = np.nan  # Ensure ydir is NaN where xdir is NaN

    # Construct the transformation matrix for body-to-global mapping
    r_roots = np.stack([xdir, ydir, np.tile(zdir, (nf, 1))], axis=1)
    r_roots[~valid_mask] = np.nan  # Set entire matrix to NaN for invalid frames
    r_roots = np.transpose(r_roots, (0, 2, 1))

    # 2. NaN handling for non-projected body rotation
    xdir_no_proj = np.column_stack((spineF_x - tailB_x, spineF_y - tailB_y, spineF_z - tailB_z))
    xdir_no_proj[~valid_mask] = np.nan  # Set NaN where invalid
    ll_no_proj = np.linalg.norm(xdir_no_proj, axis=1, keepdims=True)
    valid_mask_no_proj = (ll_no_proj[:, 0] >= 0.001)
    xdir_no_proj[~valid_mask_no_proj] = np.nan  # Propagate NaN if length is too short
    xdir_no_proj[valid_mask_no_proj] /= ll_no_proj[valid_mask_no_proj]

    ydir_no_proj = np.column_stack((-xdir_no_proj[:, 1], xdir_no_proj[:, 0], np.zeros(nf)))
    ydir_no_proj[~valid_mask_no_proj] = np.nan  # Ensure NaN in ydir
    ydir_no_proj /= np.linalg.norm(ydir_no_proj, axis=1, keepdims=True)

    zdir_no_proj = np.cross(xdir_no_proj, ydir_no_proj)
    zdir_no_proj[~valid_mask_no_proj] = np.nan  # Ensure zdir gets NaNs
    zdir_no_proj /= np.linalg.norm(zdir_no_proj, axis=1, keepdims=True)
    r_root_inv = np.stack([xdir_no_proj, ydir_no_proj, zdir_no_proj], axis=1)
    r_root_inv[~valid_mask_no_proj] = np.nan

    # 3. NaN handling for planar body direction
    xdir_planar = np.column_stack((spineF_x - tailB_x, spineF_y - tailB_y, np.zeros(nf)))
    xdir_planar[~valid_mask] = np.nan
    ll_planar = np.linalg.norm(xdir_planar, axis=1, keepdims=True)
    valid_mask_planar = (ll_planar[:, 0] >= 0.001)
    xdir_planar[~valid_mask_planar] = np.nan  # Propagate NaNs where length is too short
    xdir_planar[valid_mask_planar] /= ll_planar[valid_mask_planar]

    ydir_planar = np.column_stack((-xdir_planar[:, 1], xdir_planar[:, 0], np.zeros(nf)))
    ydir_planar[~valid_mask_planar] = np.nan
    ydir_planar /= np.linalg.norm(ydir_planar, axis=1, keepdims=True)

    zdir_fixed = np.tile([0, 0, 1], (nf, 1))
    r_root_inv_oriented = np.stack([xdir_planar, ydir_planar, zdir_fixed], axis=1)
    r_root_inv_oriented[~valid_mask_planar] = np.nan

    # 4. NaN handling for direction to mid-spine
    valid_mid_spine = ~np.isnan(spineM_x)
    dir_to_butt = np.column_stack((spineF_x - spineM_x, spineF_y - spineM_y, spineF_z - spineM_z))
    dir_to_butt[~valid_mid_spine] = np.nan
    ll_butt = np.linalg.norm(dir_to_butt, axis=1, keepdims=True)
    valid_butt_mask = (ll_butt[:, 0] >= 0.001)
    dir_to_butt[~valid_butt_mask] = np.nan  # Propagate NaNs if length is too short
    dir_to_butt[valid_butt_mask] /= ll_butt[valid_butt_mask]

    dir_to_butt = np.einsum('ijk,ik->ij', r_root_inv, dir_to_butt)
    dir_backs = dir_to_butt

    return r_roots, r_root_inv_oriented, r_root_inv, dir_backs



# Generate test data (or load real data if available)
nf = 100  # Number of frames
num_keypoints = 8  # Adjust this to match the actual number of keypoints

# Simulate random sorted_point_data for the original function
np.random.seed(0)
sorted_point_data = np.random.randn(nf, num_keypoints, 3)  # (frames, keypoints, coordinates)
sorted_point_data[::10, 4:, :] = np.nan  # Add some NaNs to simulate missing data

# Convert sorted_point_data to DataFrame format for the revised function
columns = [f"kp{i}_{axis}" for i in range(num_keypoints) for axis in ['x', 'y', 'z']]
data_flattened = sorted_point_data.reshape(nf, -1)
df_test = pd.DataFrame(data_flattened, columns=columns)

# Run both functions
r_roots_orig, r_root_inv_oriented_orig, r_root_inv_orig, dir_backs_orig = get_body_rotations(sorted_point_data)
r_roots_rev, r_root_inv_oriented_rev, r_root_inv_rev, dir_backs_rev = get_body_rotations_df_new(df_test)

# Comparison function with debug info
def compare_and_debug(name, mat1, mat2, tolerance=1e-6):
    """Compare two matrices and print debug info."""
    if mat1.shape != mat2.shape:
        print(f"Shape mismatch for {name}: {mat1.shape} vs {mat2.shape}")
        return False

    # Check element-wise differences
    diff = np.abs(mat1 - mat2)
    max_diff = np.nanmax(diff)
    max_diff_idx = np.unravel_index(np.nanargmax(diff), diff.shape)

    # Check if within tolerance
    within_tolerance = np.allclose(mat1, mat2, atol=tolerance, equal_nan=True)
    
    if not within_tolerance:
        print(f"Mismatch found in {name}:")
        print(f"  Max absolute difference: {max_diff} at index {max_diff_idx}")
        print(f"  Value in original: {mat1[max_diff_idx]}")
        print(f"  Value in revised: {mat2[max_diff_idx]}")
        print(f"  Difference array (first 5 non-zero diffs):")
        nonzero_diff_indices = np.argwhere(~np.isclose(mat1, mat2, atol=tolerance, equal_nan=True))
        for idx in nonzero_diff_indices[:5]:
            idx_tuple = tuple(idx)
            print(f"    Index: {idx_tuple}, Original: {mat1[idx_tuple]}, Revised: {mat2[idx_tuple]}, Diff: {diff[idx_tuple]}")
    else:
        print(f"{name} matches within tolerance.")
    
    return within_tolerance

# Run comparisons with debugging
results = {
    "r_roots": compare_and_debug("r_roots", r_roots_orig, r_roots_rev),
    "r_root_inv_oriented": compare_and_debug("r_root_inv_oriented", r_root_inv_oriented_orig, r_root_inv_oriented_rev),
    "r_root_inv": compare_and_debug("r_root_inv", r_root_inv_orig, r_root_inv_rev),
    "dir_backs": compare_and_debug("dir_backs", dir_backs_orig, dir_backs_rev),
}

print("\nFinal Validation Results:", results)


In [4]:
import numpy as np
import pandas as pd

# Number of frames (time steps) and number of keypoints
nf = 100
num_keypoints = 8

# Seed for reproducibility
np.random.seed(0)

# Simulate random 3D tracking data with shape (frames, keypoints, 3).
# We'll call it sorted_point_data for consistency with the original script.
sorted_point_data = np.random.randn(nf, num_keypoints, 3)

# Add some NaNs to simulate missing data (like keypoint #4+ is missing every 10th frame)
sorted_point_data[::10, 4:, :] = np.nan

###############################################################################
# For the ORIGINAL script's "data_generator", we need a dictionary
# that has all the keys it expects (even if we just put fake placeholders).
###############################################################################
old_data_dict = {
    'file_info': 'fake_file',
    'pointdatadimensions': np.array([num_keypoints, 4, nf]),  # e.g. [nMarkers, ?, nFrames]
    'trackingTS': np.arange(nf),    # pretend frame times
    'sessionTS': np.array([0, nf]), # minimal
    'ratcamTS': np.arange(nf),      # minimal
    'bbtransXY': np.zeros(nf),      # placeholders
    'bbscaleXY': np.ones(nf),       # placeholders
    'bbrot': np.zeros(nf),          # placeholders

    # The original data_generator needs "headX", "headZ", "sorted_point_data", etc.
    # We'll just guess that headX, headZ might be some subset of the data or random.
    'headorigin': np.zeros((nf,3)), # placeholder
    'headX': np.random.randn(nf, 3),
    'headZ': np.random.randn(nf, 3),
    'sorted_point_data': sorted_point_data,

    # Frame rate
    'framerate': np.array([30.0]),   # or a single float
    # If multi-session, you could do e.g. np.array([30.0, 30.0]) and add 'session_indicator'

    # No spikes in this example
}

###############################################################################
# For the REVISED DataFrame-based function, let's reshape sorted_point_data
# into a DataFrame named df_test with columns like:
# "kp0_x", "kp0_y", "kp0_z", "kp1_x", "kp1_y", ...
###############################################################################
columns = []
for i in range(num_keypoints):
    for axis in ['x','y','z']:
        columns.append(f"kp{i}_{axis}")

# Flatten the (frames, keypoints, 3) array into (frames, keypoints*3)
data_flattened = sorted_point_data.reshape(nf, -1)
df_test = pd.DataFrame(data_flattened, columns=columns)


In [3]:
pointdata.shape


(8, 4, 100)

In [1]:
import optitracking_lib as orig
import adapted_try as dfrev
import numpy as np
import pandas as pd

########################################################################
# 1) Generate Synthetic Data
########################################################################
nf = 100
num_keypoints = 8
np.random.seed(0)
sorted_point_data = np.random.randn(nf, num_keypoints, 3)
sorted_point_data[::10, 4:, :] = np.nan  # add some NaNs

########################################################################
# 2) Create 'pointdata' in shape (nMarkers, 4, nFrames).
#    The first 3 columns are x,y,z, and the 4th is the keypoint label.
########################################################################
pointdata = np.zeros((num_keypoints, 4, nf), dtype=float)
for t in range(nf):
    for kp in range(num_keypoints):
        pointdata[kp, 0, t] = sorted_point_data[t, kp, 0]  # X
        pointdata[kp, 1, t] = sorted_point_data[t, kp, 1]  # Y
        pointdata[kp, 2, t] = sorted_point_data[t, kp, 2]  # Z
        pointdata[kp, 3, t] = kp                           # label

pointdatadimensions = np.array([num_keypoints, 4, nf])  # shape

########################################################################
# 3) Prepare the old-style dictionary for original_script's data_generator
########################################################################
old_data_dict = {
    'file_info': 'fake_file',
    # The original code expects 'pointdatadimensions' and 'pointdata'
    'pointdatadimensions': pointdatadimensions,
    'pointdata': pointdata,

    # The rest are placeholders
    'trackingTS': np.arange(nf),
    'sessionTS': np.array([0, nf]),
    'ratcamTS': np.arange(nf),
    'bbtransXY': np.zeros(nf),
    'bbscaleXY': np.ones(nf),
    'bbrot': np.zeros(nf),

    # In the original script, 'headX', 'headZ', and 'sorted_point_data'
    # also get used, so let's provide them (though the code will
    # re-calculate them from 'pointdata' anyway):
    'headorigin': np.zeros((nf,3)),
    'headX': np.random.randn(nf, 3),
    'headZ': np.random.randn(nf, 3),
    'sorted_point_data': np.full((nf, num_keypoints, 3), np.nan),  # a dummy placeholder

    # Frame rate
    'framerate': np.array([30.0]),
}

########################################################################
# 4) Create a DataFrame for the revised script
########################################################################
columns = []
for i in range(num_keypoints):
    for axis in ['x','y','z']:
        columns.append(f"kp{i}_{axis}")

data_flattened = sorted_point_data.reshape(nf, -1)
df_test = pd.DataFrame(data_flattened, columns=columns)

# # Step 2: Convert df_test back to sorted_point_data format
# reconstructed_data = df_test.values.reshape(nf, num_keypoints, 3)

# # Step 3: Compare original and reconstructed data
# def compare_data(arr1, arr2, tolerance=1e-6):
#     """Compare two arrays for equality within a tolerance, including NaN values."""
#     return np.allclose(arr1, arr2, atol=tolerance, equal_nan=True)

# # Perform the comparison
# comparison_result = compare_data(sorted_point_data, reconstructed_data)

# # Output the result
# print("Is df_test correctly representing sorted_point_data?", comparison_result)

########################################################################
# 5) Now call each script
########################################################################
# Call original_script's data_generator
orig_results = orig.data_generator(old_data_dict)

# Call df_script's revised function
df_results = dfrev.process_tracking_data_df(
    df_test,
    spineF_kp=4,   # example keypoint indices
    tailB_kp=6,
    spineM_kp=5,
    headX_kp=1,    # if that is your "head X"
    headZ_kp=2,
    frame_rate=30.0
)

########################################################################
# 6) Compare some outputs
########################################################################
print("=== Original Script Results ===")
print("Keys:", list(orig_results.keys()))

print("\n=== Revised Script Results ===")
print("Keys:", list(df_results.keys()))

# Example: Compare 'body_direction' from original (in degrees or rad?)
# The original typically returns degrees for 'body_direction'.
print("\nFirst 5 of original body_direction:", orig_results['body_direction'][:5])
print("First 5 of revised body_direction_deg:", df_results['body_direction_deg'][:5])


No spikes info in the data !!! This processing will not stop, but please make sure you do not need cell data !
IF you need cell data, please press Ctrl + C to stop this processing.
Processing re-format the original data ...... 
('HEAD X is NON-NAN for', 100, 'of', 100, 'bins')
Processing to get body related rotation matrix ...... 
processing to get opt rotation .....
Optimization terminated successfully.
         Current function value: 0.000019
         Iterations: 43
         Function evaluations: 84
('opt rotation', array([-0.25464413, -0.14055001]), 'max possible rotation', 1.5707963267948966)
Processing to get head related rotation matrix ...... 
Processing to get all Euler angles ...... 
Processing to get body-related rotation matrices from DataFrame ...
Processing head rotations from DataFrame...
Computing Euler angles from the rotation matrices...
=== Original Script Results ===
Keys: ['file_info', 'settings', 'session_indicator', 'frame_times', 'full_tracking_ts', 'framerate',

/home/lq53/mir_repos/BBOP/random_tests/mini_dannce_sync/stepback_vis/tuning_clone/optitracking_lib.py:472: OptimizeWarning: Unknown solver options: xtol
  res = minimize(distance_to_xaxis, np.array([-0.25 * math.pi + rand() * 0.5 * math.pi, -0.25 * math.pi + rand() * 0.5 * math.pi]),


In [4]:
import optitracking_lib as orig
import adapted_try as dfrev
import numpy as np
import pandas as pd

########################################################################
# 1) Generate Synthetic Data
########################################################################
nf = 100
num_keypoints = 8
np.random.seed(0)
sorted_point_data = np.random.randn(nf, num_keypoints, 3)
sorted_point_data[::10, 4:, :] = np.nan  # add some NaNs

########################################################################
# 2) Create 'pointdata' in shape (nMarkers, 4, nFrames).
#    The first 3 columns are x,y,z, and the 4th is the keypoint label.
########################################################################
pointdata = np.zeros((num_keypoints, 4, nf), dtype=float)
for t in range(nf):
    for kp in range(num_keypoints):
        pointdata[kp, 0, t] = sorted_point_data[t, kp, 0]  # X
        pointdata[kp, 1, t] = sorted_point_data[t, kp, 1]  # Y
        pointdata[kp, 2, t] = sorted_point_data[t, kp, 2]  # Z
        pointdata[kp, 3, t] = kp                           # label

pointdatadimensions = np.array([num_keypoints, 4, nf])  # shape

########################################################################
# 3) Prepare the old-style dictionary for original_script's data_generator
########################################################################
old_data_dict = {
    'file_info': 'fake_file',
    # The original code expects 'pointdatadimensions' and 'pointdata'
    'pointdatadimensions': pointdatadimensions,
    'pointdata': pointdata,

    # The rest are placeholders
    'trackingTS': np.arange(nf),
    'sessionTS': np.array([0, nf]),
    'ratcamTS': np.arange(nf),
    'bbtransXY': np.zeros(nf),
    'bbscaleXY': np.ones(nf),
    'bbrot': np.zeros(nf),

    # In the original script, 'headX', 'headZ', and 'sorted_point_data'
    # also get used, so let's provide them (though the code will
    # re-calculate them from 'pointdata' anyway):
    'headorigin': np.zeros((nf,3)),
    'headX': np.random.randn(nf, 3),
    'headZ': np.random.randn(nf, 3),
    'sorted_point_data': np.full((nf, num_keypoints, 3), np.nan),  # a dummy placeholder

    # Frame rate
    'framerate': np.array([30.0]),
}

########################################################################
# 4) Create a DataFrame for the revised script
########################################################################
columns = []
for i in range(num_keypoints):
    for axis in ['x','y','z']:
        columns.append(f"kp{i}_{axis}")

data_flattened = sorted_point_data.reshape(nf, -1)
df_test = pd.DataFrame(data_flattened, columns=columns)

# Step 2: Convert df_test back to sorted_point_data format
reconstructed_data = df_test.values.reshape(nf, num_keypoints, 3)

# Step 3: Compare original and reconstructed data
def compare_data(arr1, arr2, tolerance=1e-6):
    """Compare two arrays for equality within a tolerance, including NaN values."""
    return np.allclose(arr1, arr2, atol=tolerance, equal_nan=True)

# Perform the comparison
comparison_result = compare_data(sorted_point_data, reconstructed_data)

# Output the result
print("Is df_test correctly representing sorted_point_data?", comparison_result)
    

Is df_test correctly representing sorted_point_data? True
